In [1]:
import sys
sys.path.append("../")

from PIL import Image

import torch
import torchvision
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset

from relic import ReLIC
from encoders import resnet18, resnet50
from aug import get_relic_aug_inference, get_relic_aug

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
transform = get_relic_aug_inference()

train_ds = torchvision.datasets.STL10("../data",
                                  split='train',
                                  transform=transform,
                                  download=True)
val_ds = torchvision.datasets.STL10("../data",
                                  split='test',
                                  transform=transform,
                                  download=True)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
len(train_ds), len(val_ds)

(5000, 8000)

In [6]:
ckpt = torch.load("../models/relic_model.pth")

# encoder = resnet18()
encoder = resnet50()
model = ReLIC(encoder, 128, 512)
model.load_state_dict(ckpt)

model = model.eval().to(device)

In [7]:
from tqdm.auto import tqdm

def get_embs_targets(ds):
    idx = 0
    embs, targets = [], []
    for idx, (image, target) in enumerate(tqdm(ds)):
        with torch.no_grad():
            image = image.unsqueeze(0).to(device)
            out = model.get_features(image)
            features = out[0].cpu().detach().tolist()   
            embs.append(features)
            targets.append(target)
    return embs, targets

In [8]:
embs, targets = get_embs_targets(train_ds)
embs_val, targets_val = get_embs_targets(val_ds)

  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 0/8000 [00:00<?, ?it/s]

In [9]:
import numpy as np

embeddings = np.array(embs)
labels = np.array(targets)
embeddings_val = np.array(embs_val)
labels_val = np.array(targets_val)

In [10]:
print(embeddings.shape)
print(labels.shape)
print(embeddings_val.shape)
print(labels_val.shape)

(5000, 2048)
(5000,)
(8000, 2048)
(8000,)


### Resnet 50

In [10]:
from sklearn.svm import LinearSVC, SVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
 
X_train, X_test = embeddings, embeddings_val
y_train, y_test = labels, labels_val

print("train", X_train.shape[0], len(y_train))
print("test", X_test.shape[0], len(y_test))
 
clf = LogisticRegression(max_iter=100)
clf = CalibratedClassifierCV(clf)
 
clf.fit(X_train, y_train)
 
y_pred = clf.predict(X_test)
 
acc = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)
 
print("Accuracy: ", acc)
print("Confusion matrix: \n", conf_matrix)
print("Classification report: \n", class_report)
 
y_pred_train = clf.predict(X_train)
class_report = classification_report(y_train, y_pred_train)
print("Classification report train: \n", class_report)

train 5000 5000
test 8000 8000


/home/wavelet/projects/relic/.env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/wavelet/projects/relic/.env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_

Accuracy:  0.751625
Confusion matrix: 
 [[713  22  11   3   2   8   2   0  24  15]
 [ 28 590   1  49  27  25   9  66   3   2]
 [  9   1 742   3   0   3   3   0   6  33]
 [  1  63   0 493  70  90  19  62   2   0]
 [  4  41   2  39 567  28  92  24   3   0]
 [  0  38   3 114  63 372 125  82   1   2]
 [  5   6   5   9  33  32 672  29   5   4]
 [  0  72   1  60  38  61  41 525   2   0]
 [ 50   3   7   3   0   1   0   0 705  31]
 [ 32   0  55   8   0   2   4   3  62 634]]
Classification report: 
               precision    recall  f1-score   support

           0       0.85      0.89      0.87       800
           1       0.71      0.74      0.72       800
           2       0.90      0.93      0.91       800
           3       0.63      0.62      0.62       800
           4       0.71      0.71      0.71       800
           5       0.60      0.47      0.52       800
           6       0.69      0.84      0.76       800
           7       0.66      0.66      0.66       800
           8     

### Resnet 18

In [10]:
from sklearn.svm import LinearSVC, SVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
 
X_train, X_test = embeddings, embeddings_val
y_train, y_test = labels, labels_val

print("train", X_train.shape[0], len(y_train))
print("test", X_test.shape[0], len(y_test))
 
clf = LogisticRegression(max_iter=100)
clf = CalibratedClassifierCV(clf)
 
clf.fit(X_train, y_train)
 
y_pred = clf.predict(X_test)
 
acc = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)
 
print("Accuracy: ", acc)
print("Confusion matrix: \n", conf_matrix)
print("Classification report: \n", class_report)
 
y_pred_train = clf.predict(X_train)
class_report = classification_report(y_train, y_pred_train)
print("Classification report train: \n", class_report)

train 5000 5000
test 8000 8000


/home/wavelet/projects/relic/.env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/wavelet/projects/relic/.env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_

Accuracy:  0.723875
Confusion matrix: 
 [[669  43  19  10   2   3   4   1  35  14]
 [ 17 543   1  68  18  39  13 100   0   1]
 [ 16   4 718  12   1   0   6   1  10  32]
 [  0  70   5 479  69  90  14  67   4   2]
 [  2  36   1  58 559  35  70  36   1   2]
 [  2  38   5 109  54 380 108 101   0   3]
 [  3   9   6  20  38  53 635  29   4   3]
 [  1  68   2  70  28  58  34 536   1   2]
 [ 38   8  17   6   1   1   0   0 702  27]
 [ 23   6  97  13   0   4  12   3  72 570]]
Classification report: 
               precision    recall  f1-score   support

           0       0.87      0.84      0.85       800
           1       0.66      0.68      0.67       800
           2       0.82      0.90      0.86       800
           3       0.57      0.60      0.58       800
           4       0.73      0.70      0.71       800
           5       0.57      0.47      0.52       800
           6       0.71      0.79      0.75       800
           7       0.61      0.67      0.64       800
           8     

/home/wavelet/projects/relic/.env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
